<a href="https://colab.research.google.com/github/Rohit-Ray-Git/AI-Powered-Document-Insight-Assistant/blob/main/site/en/gemini-api/docs/vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Explore vision capabilities with the Gemini API

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemini-api/docs/vision"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />View on ai.google.dev</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/vision.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

The Gemini API is able to process images and videos, enabling a multitude of
 exciting developer use cases. Some of Gemini's vision capabilities include
 the ability to:

*   Caption and answer questions about images
*   Transcribe and reason over PDFs, including long documents up to 2 million token context window
*   Describe, segment, and extract information from videos,
including both visual frames and audio, up to 90 minutes long
*   Detect objects in an image and return bounding box coordinates for them

This tutorial demonstrates some possible ways to prompt the Gemini API with
images and video input, provides code examples,
and outlines prompting best practices with multimodal vision capabilities.
All output is text-only.

## Setup

Before you use the File API, you need to install the Gemini API SDK package and configure an API key. This section describes how to complete these setup steps.

### Install the Python SDK and import packages

The Python SDK for the Gemini API is contained in the [google-generativeai](https://pypi.org/project/google-generativeai/) package. Install the dependency using pip.

In [5]:
!pip install -q -U google-generativeai

Import the necessary packages.

In [6]:
import google.generativeai as genai
from IPython.display import Markdown

### Set up your API key

The File API uses API keys for authentication and access. Uploaded files are associated with the project linked to the API key. Unlike other Gemini APIs that use API keys, your API key also grants access to data you've uploaded to the File API, so take extra care in keeping your API key secure. For more on keeping your keys
secure, see [Best practices for using API
keys](https://support.google.com/googleapi/answer/6310037).

Store your API key in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or are unfamiliar with Colab Secrets, refer to the [Authentication quickstart](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb).

In [7]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Prompting with images

In this tutorial, you will upload images using the File API or as inline data and generate content based on those images.

### Technical details (images)
Gemini 1.5 Pro and Flash support a maximum of 3,600 image files.

Images must be in one of the following image data [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):

-   PNG - `image/png`
-   JPEG - `image/jpeg`
-   WEBP - `image/webp`
-   HEIC - `image/heic`
-   HEIF - `image/heif`

Each image is equivalent to 258 tokens.

While there are no specific limits to the number of pixels in an image besides the model’s context window, larger images are scaled down to a maximum resolution of 3072x3072 while preserving their original aspect ratio, while smaller images are scaled up to 768x768 pixels. There is no cost reduction for images at lower sizes, other than bandwidth, or performance improvement for images at higher resolution.

For best results:

*   Rotate images to the correct orientation before uploading.
*   Avoid blurry images.
*   If using a single image, place the text prompt after the image.

## Image input

For total image payload size less than 20MB, it's recommended to either upload
base64 encoded images or directly upload locally stored image files.

### Base64 encoded images

You can upload public image URLs by encoding them as Base64 payloads.
You can use the httpx library to fetch the image URLs.
The following code example shows how to do this:

In [8]:
import httpx
import base64

# Retrieve an image
image_path = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image = httpx.get(image_path)

# Choose a Gemini model
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Create a prompt
prompt = "Caption this image."
response = model.generate_content(
    [
        {
            "mime_type": "image/jpeg",
            "data": base64.b64encode(image.content).decode("utf-8"),
        },
        prompt,
    ]
)

Markdown(">" + response.text)

>A view of London, England, showcasing the city's iconic landmarks under a cloudy sky.  Prominently featured are the London Eye, Big Ben (Elizabeth Tower), the Houses of Parliament, and a glimpse of the Shard in the background.  The image captures the mix of historical and modern architecture, along with the River Thames and the bustling cityscape. The viewpoint is elevated, providing a broad perspective over the rooftops and streets below.

### Multiple images

To prompt with multiple images in Base64 encoded format, you can do the
following:

In [9]:
import httpx
import base64

# Retrieve two images
image_path_1 = "https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg/2560px-Palace_of_Westminster_from_the_dome_on_Methodist_Central_Hall.jpg"
image_path_2 = "https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg"

image_1 = httpx.get(image_path_1)
image_2 = httpx.get(image_path_2)

# Create a prompt
prompt = "Generate a list of all the objects contained in both images."

response = model.generate_content([
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_1.content).decode('utf-8')},
{'mime_type':'image/jpeg', 'data': base64.b64encode(image_2.content).decode('utf-8')}, prompt])

Markdown(response.text)

**Image 1 (London Skyline):**

* Houses of Parliament (including Big Ben/Elizabeth Tower)
* London Eye
* The Shard
* River Thames
* Various other buildings and skyscrapers of London
* Trees
* Streets
* Construction cranes

**Image 2 (Jetpack Backpack Sketch):**

* Backpack shape
* Padded strap support
* USB-C charging port
* Retractible boosters with steam exhaust
* Text labels indicating features:
    * "JETPACK BACKPACK"
    * "FITS 18" LAPTOP"
    * "LIGHTWEIGHT"
    * "LOOKS LIKE A NORMAL BACKPACK"
    * "RETRACTABLE BOOSTERS"
    * "STEAM-POWERED, GREEN/CLEAN"
    * "PADDED STRAP SUPPORT"
    * "USB-C CHARGING"
    * "15-MIN BATTERY LIFE"

### Upload one or more locally stored image files

Alternatively, you can upload one or more locally stored image files..

You can download and use our drawings of [piranha-infested waters](https://storage.googleapis.com/generativeai-downloads/images/piranha.jpg) and a [firefighter with a cat](https://storage.googleapis.com/generativeai-downloads/images/firefighter.jpg). First, save these files to your local directory.

Then click **Files** on the left sidebar. For each file, click the **Upload** button, then navigate to that file's location and upload it:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files. Smaller files can instead be called locally from the Gemini API:


In [12]:
import PIL.Image

sample_file_2 = PIL.Image.open('img1.jpg')
sample_file_3 = PIL.Image.open('img2.jpeg')

In [13]:
import google.generativeai as genai

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt.
prompt = "Write an advertising jingle based on the items in both images."

response = model.generate_content([sample_file_2, sample_file_3, prompt])

Markdown(response.text)

(Upbeat, samba-style music plays)

**Singer:** Feeling the heat? Like a piranha's bite?
House on fire, burning bright?
Don't delay, don't you fret!
Call the firemen, haven't you met?

**(Tempo changes to a more heroic march)**

**Singer:**  Brave and strong, fast and bold,
Putting out fires, young and old!
From flames so high, to embers low,
They'll stop the blaze, put on a show!

**(Music returns to samba)**

**Singer:** So when your house is feeling hot,
Like a piranha in a pot,
Don't get burned, don't despair,
Call the fire department, they'll be there!

**(Announcer, clear and confident):**  For all your fire emergencies, call 9-1-1.  Your safety is our priority!


Note that these inline data calls don't include many of the features available
through the File API, such as getting file metadata,
[listing](https://ai.google.dev/gemini-api/docs/vision?lang=python#list-files),
or [deleting files](https://ai.google.dev/gemini-api/docs/vision?lang=python#delete-files).

### Large image payloads

#### Upload an image file using the File API

When the combination of files and system instructions that you intend to send is larger than 20 MB in size, use the File API to upload those files.

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but cannot be downloaded from the API. It is available at no cost in all regions where the Gemini API is available.

Upload the image using [`media.upload`](https://ai.google.dev/api/rest/v1beta/media/upload) and print the URI, which is used as a reference in Gemini API calls.

In [14]:
!curl -o jetpack.jpg https://storage.googleapis.com/generativeai-downloads/images/jetpack.jpg

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  349k  100  349k    0     0  2638k      0 --:--:-- --:--:-- --:--:-- 2645k


In [15]:
# Upload the file and print a confirmation.
sample_file = genai.upload_file(path="jetpack.jpg",
                            display_name="Jetpack drawing")

print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")

Uploaded file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/yps21jkbw1i7


The `response` shows that the File API stored the specified `display_name` for the uploaded file and a `uri` to reference the file in Gemini API calls. Use `response` to track how uploaded files are mapped to URIs.

Depending on your use case, you can also store the URIs in structures such as a `dict` or a database.

#### Verify image file upload and get metadata

You can verify the API successfully stored the uploaded file and get its metadata by calling [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) through the SDK. Only the `name` (and by extension, the `uri`) are unique. Use `display_name` to identify files only if you manage uniqueness yourself.

In [16]:
file = genai.get_file(name=sample_file.name)
print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")

Retrieved file 'Jetpack drawing' as: https://generativelanguage.googleapis.com/v1beta/files/yps21jkbw1i7


Depending on your use case, you can store the URIs in structures, such as a `dict` or a database.

#### Prompt with the uploaded image and text

After uploading the file, you can make GenerateContent requests that reference the File API URI. Select the generative model and provide it with a text prompt and the uploaded image.

In [17]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Prompt the model with text and the previously uploaded image.
response = model.generate_content([sample_file, "Describe how this product might be manufactured."])

Markdown(response.text)

Manufacturing a jetpack backpack, even a conceptual steam-powered one, would require a multi-stage process involving several specialized areas:

1. **Backpack Construction:** This would follow standard backpack manufacturing techniques.
    * **Fabric Cutting and Sewing:**  Durable, lightweight, and possibly fire-resistant fabrics (nylon, Cordura, etc.) would be cut and sewn to form the main compartment, straps (with added padding and reinforcement for the jetpack thrust), and other compartments.
    * **Hardware:**  Buckles, zippers, and other hardware would be attached.

2. **Jetpack System Fabrication:** This is where specialized manufacturing comes in.
    * **Booster Construction:**  Small, retractable steam-powered boosters would need to be engineered. This might involve miniature boilers, valves, and nozzles, likely using lightweight, heat-resistant metals like titanium or specialized alloys. Precision machining and welding would be essential.
    * **Steam Generation:** A compact, lightweight steam generator would be needed. This could involve a miniaturized boiler heated by a powerful, compact energy source (perhaps a high-density battery, though not explicitly mentioned in the drawing). This requires expertise in thermodynamics, materials science, and miniaturization.
    * **Control System:**  A system of valves and controls (possibly electronic) would regulate steam flow to the boosters for controlled thrust. This involves electronics manufacturing and software development for the control system.

3. **Power Source Production:**  If using batteries, a specialized battery manufacturing process would be required for high energy density and rapid discharge capabilities.

4. **Assembly and Testing:**
    * **Integration:**  The backpack and jetpack components would be carefully integrated and secured. This would involve fitting the steam generator, boosters, and control system within the backpack structure while maintaining balance and user comfort.
    * **Quality Control:**  Rigorous testing would be essential. This would include pressure tests for the steam system, thrust tests for the boosters, flight stability tests, and safety checks for the entire system.  Given the nature of a jetpack, certification and regulatory compliance would be critical before market release.

5. **USB-C Charging System Integration:**  Standard USB-C charging ports and circuitry would be integrated for charging the power source (presumably the battery that powers the steam generator).

This is a simplified overview. A real-world jetpack backpack would require much more complex engineering and manufacturing processes. The steam-powered aspect adds further complexity, as steam power is not typically used for personal flight devices due to efficiency and safety concerns.


## Capabilties

This section outlines specific vision capabilities of the Gemini model, including object detection and bounding box coordinates.

### Get bounding boxes

Gemini models are trained to return bounding box coordinates as relative widths or heights in the range of [0, 1]. These values are then scaled by 1000 and converted to integers. Effectively, the coordinates represent the bounding box on a 1000x1000 pixel version of the image. Therefore, you'll need to convert these coordinates back to the dimensions of your original image to accurately map the bounding boxes.

In [18]:
# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Create a prompt to detect bounding boxes.
prompt = "Return a bounding box for each of the objects in this image in [ymin, xmin, ymax, xmax] format."
response = model.generate_content([sample_file_2, prompt])

Markdown(response.text)

I detect the following object and its bounding box.
- 275 159 729 841 fish


The model returns bounding box coordinates in the format
`[ymin, xmin, ymax, xmax]`. To convert these normalized coordinates
to the pixel coordinates of your original image, follow these steps:

1.    Divide each output coordinate by 1000.
1.    Multiply the x-coordinates by the original image width.
1.    Multiply the y-coordinates by the original image height.

To explore more detailed examples of generating bounding box coordinates and
visualizing them on images, review our [Object Detection cookbook example](https://github.com/google-gemini/cookbook/blob/main/examples/Object_detection.ipynb).

## Prompting with video

In this tutorial, you will upload a video using the File API and generate content based on those images.

### Technical details (video)

Gemini 1.5 Pro and Flash support up to approximately an hour of video data.

Video must be in one of the following video format [MIME types](https://developers.google.com/drive/api/guides/ref-export-formats):
  -   `video/mp4`
  -   `video/mpeg`
  -   `video/mov`
  -   `video/avi`
  -   `video/x-flv`
  -   `video/mpg`
  -   `video/webm`
  -   `video/wmv`
  -   `video/3gpp`

The File API service currently extracts image frames from videos at 1 frame per second (FPS) and audio at 1Kbps, single channel, adding timestamps every second. These rates are subject to change in the future for improvements in inference.

**NOTE:** The finer details of fast action sequences may be lost at the 1 FPS frame sampling rate. Consider slowing down high-speed clips for improved inference quality.

Individual frames are 258 tokens, and audio is 32 tokens per second. With metadata, each second of video becomes ~300 tokens, which means a 1M context window can fit slightly less than an hour of video.

To ask questions about time-stamped locations, use the format `MM:SS`, where the first two digits represent minutes and the last two digits represent seconds.

For best results:

*   Use one video per prompt.
*   If using a single video, place the text prompt after the video.

### Upload a video file to the File API

**NOTE**: The File API lets you store up to 20 GB of files per project, with a per-file maximum size of 2 GB. Files are stored for 48 hours. They can be accessed in that period with your API key, but they cannot be downloaded using any API. It is available at no cost in all regions where the Gemini API is available.

The File API accepts video file formats directly. This example uses the short NASA film ["Jupiter's Great Red Spot Shrinks and Grows"](https://www.youtube.com/watch?v=JDi4IdtvDVE0). Credit: Goddard Space Flight Center (GSFC)/David Ladd (2018).

> "Jupiter's Great Red Spot Shrinks and Grows" is in the public domain and does not show identifiable people. ([NASA image and media usage guidelines.](https://www.nasa.gov/nasa-brand-center/images-and-media/))

Start by retrieving the short video:

In [19]:
!wget https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4

--2024-12-26 09:34:34--  https://storage.googleapis.com/generativeai-downloads/images/GreatRedSpot.mp4
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.194.207, 142.250.152.207, 172.217.214.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.194.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238090979 (227M) [video/mp4]
Saving to: ‘GreatRedSpot.mp4’

GreatRedSpot.mp4    100%[===================>] 227.06M   187MB/s    in 1.2s    

2024-12-26 09:34:36 (187 MB/s) - ‘GreatRedSpot.mp4’ saved [238090979/238090979]



Upload the video to the File API and print the URI.

In [20]:
video_file_name = "GreatRedSpot.mp4"

print(f"Uploading file...")
video_file = genai.upload_file(path=video_file_name)
print(f"Completed upload: {video_file.uri}")

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/80xqsj1d7lfg


### Verify file upload and check state

Verify the API has successfully received the files by calling the [`files.get`](https://ai.google.dev/api/rest/v1beta/files/get) method.

**NOTE**: Video files have a `State` field in the File API. When a video is uploaded, it will be in the `PROCESSING` state until it is ready for inference. Only `ACTIVE` files can be used for model inference.

In [21]:
import time

# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(10)
    video_file = genai.get_file(video_file.name)

if video_file.state.name == "FAILED":
  raise ValueError(video_file.state.name)

.

### Prompt with a video and text

Once the uploaded video is in the `ACTIVE` state, you can make `GenerateContent` requests that specify the File API URI for that video. Select the generative model and provide it with the uploaded video and a text prompt.

In [22]:
# Create the prompt.
prompt = "Summarize this video. Then create a quiz with answer key based on the information in the video."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})

# Print the response, rendering any Markdown
Markdown(response.text)

Making LLM inference request...


Here is a summary of the video and a quiz about Jupiter’s Great Red Spot:

Jupiter is the largest and oldest planet in our solar system, dating back 4.5 billion years. It’s made of the same elements as stars but never became massive enough to ignite. It’s comprised mostly of swirled gases and liquids and its iconic feature, the Great Red Spot, is a giant, centuries-old storm.  

Scientists have noticed that the Great Red Spot is evolving. The spot’s color is deepening, and it is shrinking and becoming more circular. These scientists theorized that as the storm shrinks, the wind speeds should increase, like a spinning ice skater who pulls in her arms. 

However, recent data shows that the spot isn’t rotating faster, it’s getting taller. The change is analogous to the way clay spins as it is molded on a potter’s wheel. Although it’s shrinking horizontally, the spot is increasing vertically.  

The Great Red Spot used to be large enough to hold three Earths, but it’s now just a little bigger than one. The data is based on analyses from NASA missions like Voyager, Hubble, and Juno. Scientists are hopeful that further investigation will reveal more secrets of the mysterious Great Red Spot.


Quiz:

1. Which planet is considered the oldest and largest in our solar system?
A. Mars
B. Earth
C. Jupiter
D. Saturn

2. What is the Great Red Spot?
A. A gigantic volcano
B. A giant storm
C. A giant crater
D. An impact zone

3. What two characteristics of the Great Red Spot are changing over time?
A. Size and shape
B. Temperature and depth
C. Composition and color
D. Rotation speed and visibility

4. How does the shrinking of the Great Red Spot affect its wind speed?
A. Wind speeds increase
B. Wind speeds decrease
C. Wind speeds remain constant
D. Wind speeds fluctuate randomly

5. What analogy is used to explain the change in height of the Great Red Spot?
A. A spinning top
B. A figure skater pulling in her arms
C. Molding clay on a potter's wheel
D. An inflatable balloon being deflated

Answer Key:
1. C
2. B
3. A
4. C
5. C

### Refer to timestamps in the content

You can use timestamps of the form `MM:SS` to refer to specific moments in the video.

In [23]:
# Create the prompt.
prompt = "What are the examples given at 01:05 and 01:19 supposed to show us?"

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([prompt, video_file],
                                  request_options={"timeout": 600})
Markdown(response.text)

Making LLM inference request...


Sure! At [00:00:05], the narrator explains that scientists expect that if Jupiter’s Great Red Spot shrinks, its wind speed should increase. They compare this expected outcome to the way an ice skater spins faster when she pulls in her arms. Then at [00:00:19], the video displays data from 2014 to 2017 charting relative height and latitude. While the spot is shrinking, it is also growing taller. The analogy of someone working with clay on a potter’s wheel is used to explain this phenomenon. As the large lump of clay spins on the potter’s wheel, forces are pushing it upward, making it taller. Similarly, while the Great Red Spot appears to shrink from our perspective, it is also growing taller.

### Transcribe video and provide visual descriptions

The Gemini models can transcribe and provide visual descriptions of video content
by processing both the audio track and visual frames.
For visual descriptions, the model samples the video at a rate of **1 frame
per second**. This sampling rate may affect the level of detail in the
descriptions, particularly for videos with rapidly changing visuals.

In [24]:
# Create the prompt.
prompt = "Transcribe the audio from this video, giving timestamps for salient events in the video. Also provide visual descriptions."

# Choose a Gemini model.
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest")

# Make the LLM request.
print("Making LLM inference request...")
response = model.generate_content([video_file, prompt],
                                  request_options={"timeout": 600})
Markdown(esponse.text)

Making LLM inference request...


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro-latest:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

## List files

You can list all uploaded files and their URIs using `files.list_files()`.

In [25]:
# List all files
for file in genai.list_files():
    print(f"{file.display_name}, URI: {file.uri}")

GreatRedSpot.mp4, URI: https://generativelanguage.googleapis.com/v1beta/files/80xqsj1d7lfg
Jetpack drawing, URI: https://generativelanguage.googleapis.com/v1beta/files/yps21jkbw1i7


## Delete files

Files are automatically deleted after 2 days. You can also manually delete them using `files.delete()`.

In [ ]:
genai.delete_file(video_file.name)
print(f'Deleted file {video_file.uri}')